In [1]:
import numpy as np
import pandas as pd

path='../raw/'
date_list=['2021-04-13','2021-04-14','2021-04-15','2021-04-16']

meta=pd.read_csv('../meta/Foothill_detectors.csv')



In [11]:
for i in date_list:
    
    df=pd.read_csv(path+i+'.csv')
    df=df[df['intersection'].isin(meta.SignalID.unique())]
    df['time']=pd.to_datetime(df['time'])
    df=df.rename(columns={'intersection':'signal','code':'event','para':'parameter', 'time':'datetime'})

    df['datetime']=pd.to_datetime(df['datetime'])

    cycle_df=pd.read_csv('../occupancy/'+i+'/'+'cycle_df_'+i+'.csv', index_col=[0])


    cycle_df=cycle_df[cycle_df['cycle_lenght']==150]
    cycle_df=cycle_df.sort_values(['signal','start_cycle']) 
    cycle_df=cycle_df.reset_index(drop=True)



    grouped=cycle_df.groupby(['signal'])
    id_cycle_df=pd.DataFrame([])
    for sets , data in grouped:
        data=data.sort_values(['start_cycle']).reset_index(drop=True)
        data['cycle_id']=data.index
        id_cycle_df=id_cycle_df.append(data)

    id_cycle_df=id_cycle_df.sort_values(['signal','start_cycle']).reset_index(drop=True)
    id_cycle_df['unique_cycle_id']=id_cycle_df.index
    id_cycle_df.to_csv(i+'/'+'id_cycle_df'+'_'+i+'.csv')

    detector_df=pd.read_csv('../occupancy/'+i+'/'+'detector_df'+i+'.csv', index_col=[0])
    
    import sqlite3
    #Make the db in memory
    frequency_db = sqlite3.connect(':memory:')
    #write the tables
    detector_df.to_sql('detector', frequency_db, index=False)
    id_cycle_df.to_sql('cycle_df', frequency_db, index=False)

    qry = '''
    select  
        detector.*
        ,cycle_df.start_cycle
        ,cycle_df.end_cycle
        ,cycle_df.signal
        ,cycle_df.cycle_id
        ,cycle_df.unique_cycle_id
    from detector 
        join cycle_df  using(signal)
        WHERE
        detector_on_start between start_cycle and end_cycle


    '''


    df_query_frequency = pd.read_sql_query(qry, frequency_db)
    df_query_frequency = df_query_frequency.loc[:,~df_query_frequency.columns.duplicated()]

    df_query_frequency['start_cycle']=pd.to_datetime(df_query_frequency['start_cycle'])
    df_query_frequency['end_cycle']=pd.to_datetime(df_query_frequency['end_cycle'])
    df_query_frequency['detector_on_start']=pd.to_datetime(df_query_frequency['detector_on_start'])
    df_query_frequency['detector_on_end']=pd.to_datetime(df_query_frequency['detector_on_end'])


    df_query_frequency.to_csv(i+'/'+'df_query_frequency'+'_'+i+'.csv')
    df_query_frequency['start_cycle']=pd.to_datetime(df_query_frequency['start_cycle'])
    df_query_frequency['end_cycle']=pd.to_datetime(df_query_frequency['end_cycle'])
    df_query_frequency['detector_on_start']=pd.to_datetime(df_query_frequency['detector_on_start'])
    df_query_frequency['detector_on_end']=pd.to_datetime(df_query_frequency['detector_on_end'])


    df_query_frequency['start_chunk']=np.ceil((df_query_frequency['detector_on_start']-df_query_frequency['start_cycle']).dt.total_seconds()/5)
    # because of ceiling issue , when detector on time is the same as cycle start time it will give 0 we push that to 1 
    # manually below
    df_query_frequency['start_chunk']=np.where(df_query_frequency['start_chunk']==0,1,df_query_frequency['start_chunk'])

    df_query_frequency.to_csv(i+'/'+'df_query_frequency'+'_'+i+'.csv')

    f=df_query_frequency.groupby(['signal','detector_parameter','start_cycle','end_cycle','cycle_id','unique_cycle_id','start_chunk'])['unique_detector_id'].count().reset_index().sort_values(['unique_detector_id'],ascending=False)
    f=f.rename(columns={'unique_detector_id':'count'})

    f=f.merge(meta, how='left', left_on=['signal','detector_parameter'], right_on=['SignalID','DetChannel']).drop(['SignalID'],axis=1)
    f.to_csv(i+'/'+'frequency_v1'+'_'+i+'.csv')
